In [1]:
import time
from iot_mqtt import PumpMQTT, UltraMQTT, HeatMQTT, start_broker_if_needed, stop_broker
proc = start_broker_if_needed()  # only if you don’t run the Windows service

broker = "192.168.0.100"

pumps = PumpMQTT(broker=broker, username="pump1", password="pump",
                base_topic="pumps/01", client_id="pyctl-pumps")
ultra = UltraMQTT(broker=broker, username="ultra1", password="ultra",
                base_topic="ultra/01", client_id="pyctl-ultra")
heat = HeatMQTT(broker=broker, username="heat1", password="heat",
                base_topic="heat/01", client_id="pyctl-heat")

pumps.ensure_connected()
ultra.ensure_connected()
heat.ensure_connected()
time.sleep(1)  # wait for connections to settle
print("Connected to broker")

[broker] Logging to: C:\Users\13538\OneDrive\Desktop\Arduino\mosq-python.log
[broker] Starting Mosquitto (-v for logs)...
[broker] Broker is ready.
[pyctl-pumps] Connecting to 192.168.0.100:1883 (attempt 1)...
[pyctl-ultra] Connecting to 192.168.0.100:1883 (attempt 1)...
[pyctl-pumps] Connected to 192.168.0.100:1883
[pyctl-ultra] Connected to 192.168.0.100:1883
[pyctl-heat] Connecting to 192.168.0.100:1883 (attempt 1)...
[pyctl-heat] Connected to 192.168.0.100:1883
Connected to broker


pumps/01/status OFFLINE
pumps/01/state/1 OFF
pumps/01/state/2 OFF
pumps/01/state/3 OFF
pumps/01/state/4 OFF
ultra/01/status OFFLINE
ultra/01/state/1 OFF
ultra/01/state/2 OFF
heat/01/status ONLINE
heat/01/state/1 OFF
heat/01/state/2 OFF
heat/01/temp/1 23.1
pumps/01/status OFFLINE
pumps/01/state/1 OFF
pumps/01/state/2 OFF
pumps/01/state/3 OFF
pumps/01/state/4 OFF
ultra/01/status ONLINE
ultra/01/state/1 OFF
ultra/01/state/2 OFF
heat/01/status ONLINE
heat/01/state/1 OFF
heat/01/state/2 OFF
pumps/01/state/3 OFF
pumps/01/state/4 OFF
ultra/01/status ONLINE
ultra/01/state/1 OFF
ultra/01/state/2 OFF
ultra/01/heartbeat 1
heat/01/status ONLINE
heat/01/state/1 OFF
heat/01/state/2 OFF
heat/01/temp/1 23.0


In [6]:
# Show live status from each node for a couple seconds
pumps.status(seconds=2.0)
ultra.status(seconds=2.0)
heat.status(seconds=2.0)

In [5]:
# Pumps
pumps.on(1)                 # pump 1 ON
time.sleep(1)
pumps.on(1, 2000)           # pump 1 ON for 2 s (auto-off)
time.sleep(2.5)

[pyctl-pumps] Published 'ON' to pumps/01/cmd/1
[pyctl-pumps] Published 'ON:2000' to pumps/01/cmd/1


In [ ]:
# Ultrasonic
ultra.on_for(2, 30000)       # ultrasonic ch2 ON for 30 s


[pyctl-ultra] Published 'ON:30000' to ultra/01/cmd/2


In [ ]:
# Heater + thermistor demo
heat.set_target(1, 42.0)         # set target to 42C (ESP retains on set/1)
heat.pid_on(1)                   # enable PID loop on ESP
# actively request a reading now:
try:
    for i in range(200):
        t = heat.get_base_temp(1, timeout_s=5.0)
        print("Temp(ch1) =", t)
        time.sleep(1)
except TimeoutError as e:
    print("Temp read timeout:", e)

heat.pid_off(1)                  # stop PID
heat.set_pwm(1, 0)               # ensure PWM is off
heat.off(1)                      # ensure relay is off

[pyctl-heat] Published 'SET:42.0' to heat/01/cmd/1
[pyctl-heat] Published 'PID:ON' to heat/01/cmd/1
[pyctl-heat] Published 'PID:OFF' to heat/01/cmd/1
[pyctl-heat] Published 'PWM:0' to heat/01/cmd/1
[pyctl-heat] Published 'OFF' to heat/01/cmd/1


In [19]:
# Cleanup
pumps.disconnect()
ultra.disconnect()
heat.disconnect()
stop_broker(proc)

[broker] Stopping Mosquitto...
[2025-10-20 11:05:39] [broker] Mosquitto stopped.
